# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guarapari,-20.6667,-40.4975,23.83,72,81,8.37,BR,1696180716
1,1,remire-montjoly,4.9167,-52.2667,33.02,58,75,4.63,GF,1696180815
2,2,port-aux-francais,-49.3500,70.2167,6.26,97,100,19.13,TF,1696180815
3,3,palmer,42.1584,-72.3287,24.32,59,0,4.63,US,1696180815
4,4,mercedes,-34.6515,-59.4307,22.19,29,0,0.83,AR,1696180815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# # Import the required libraries
# import hvplot.pandas
# import pandas as pd

# # Configure the map plot
# hv.extension('bokeh')
# map_plot = city_data_df.hvplot.points(
#     "Lng",
#     "Lat",
#     geo = True,
#     tiles = "OSM",
#     frame_width = 700,
#     frame_height = 500,
#     size = "Humidity",
#     scale = 0.01,
#     color = "City"
# )

# # Display the map plot
# map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
city_range = city_data_df[(city_data_df['Max Temp'].between(21,27, inclusive=False))&(city_data_df['Wind Speed']>4)&(city_data_df['Cloudiness']==0)]
city_range
# Drop any rows with null values
columns_to_check = ['City_ID','City','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']
city_range_df = city_range.dropna(subset=columns_to_check)

# Display sample data
city_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,palmer,42.1584,-72.3287,24.32,59,0,4.63,US,1696180815
20,20,giado,31.9582,12.0203,21.79,55,0,6.10,LY,1696180819
42,42,dakhla,23.6848,-15.9580,25.01,69,0,8.75,EH,1696180823
116,116,pisco,-13.7000,-76.2167,21.12,78,0,4.12,PE,1696180837
138,138,vila velha,-20.3297,-40.2925,24.97,81,0,4.63,BR,1696180842
156,156,bethel,41.3712,-73.4140,24.56,65,0,4.63,US,1696180845
180,180,karratha,-20.7377,116.8463,25.55,32,0,7.28,AU,1696180850
328,328,fairview,40.8126,-73.9990,25.43,53,0,4.63,US,1696180878
351,351,forest acres,34.0193,-80.9898,26.99,57,0,6.17,US,1696180883
364,364,laramie,41.3114,-105.5911,22.75,25,0,7.72,US,1696180592


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_range_df[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,palmer,US,42.1584,-72.3287,59,NaN
20,giado,LY,31.9582,12.0203,55,NaN
42,dakhla,EH,23.6848,-15.9580,69,NaN
116,pisco,PE,-13.7000,-76.2167,78,NaN
138,vila velha,BR,-20.3297,-40.2925,81,NaN
156,bethel,US,41.3712,-73.4140,65,NaN
180,karratha,AU,-20.7377,116.8463,32,NaN
328,fairview,US,40.8126,-73.9990,53,NaN
351,forest acres,US,34.0193,-80.9898,57,NaN
364,laramie,US,41.3114,-105.5911,25,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:

import requests
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "conditions": "internet_access",
          "lang" : "en",
          "limit" : "10",
          "apikey" : geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitudes},{latitudes},{radius}"
    params["bias"] = f"proximity:{longitudes},{latitudes}"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    name_address.url
    # Convert the API response to JSON format
    name_address = requests.get(base_url,params = params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmer - nearest hotel: No hotel found
giado - nearest hotel: No hotel found
dakhla - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
fairview - nearest hotel: No hotel found
forest acres - nearest hotel: No hotel found
laramie - nearest hotel: No hotel found
farmington - nearest hotel: No hotel found
callao - nearest hotel: No hotel found
plainfield - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
santiago de cao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
3,palmer,US,42.1584,-72.3287,59,No hotel found
20,giado,LY,31.9582,12.0203,55,No hotel found
42,dakhla,EH,23.6848,-15.9580,69,No hotel found
116,pisco,PE,-13.7000,-76.2167,78,No hotel found
138,vila velha,BR,-20.3297,-40.2925,81,No hotel found
156,bethel,US,41.3712,-73.4140,65,No hotel found
180,karratha,AU,-20.7377,116.8463,32,No hotel found
328,fairview,US,40.8126,-73.9990,53,No hotel found
351,forest acres,US,34.0193,-80.9898,57,No hotel found
364,laramie,US,41.3114,-105.5911,25,No hotel found


In [13]:
name_address = requests.get(base_url,params = params)
name_address.url

'https://api.geoapify.com/v2/places?categories=accommodation.hotel&conditions=internet_access&lang=en&limit=10&apikey=56da03c3845149c2af9b85785cb97527&filter=circle%3A-79.2392%2C-7.9589%2C10000&bias=proximity%3A-79.2392%2C-7.9589'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

# # Configure the map plot
# hv.extension('bokeh')
# hotel_map_plot = hotel_df.hvplot.points(
#     "Lng",
#     "Lat",
#     geo = True,
#     tiles = "OSM",
#     frame_width = 700,
#     frame_height = 500,
#     size = "Humidity",
#     scale = 0.01,
#     color = ["City","Hotel Name","Country"]
# )


# # Display the map
# hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)